In [1]:
import numpy as np
import scipy as sp

# Define known parameters
rho1 = 1600; mu1 = 40e-3; N1 = 30000; D1 = 80e-3; M1 = 0.8; P1 = 36e5
rho2 = 1159.5; mu2 = 9.5592e-3
g = 9.81  # Acceleration due to gravity in m/s^2

Q1 = M1/rho1

# Define lower and upper bounds
lb = [2500, 30e-3, 0.1, 1e5]  # Lower bounds
ub = [5500, 150e-3, 1.0, 15e5]  # Upper bounds
bounds = [(low, high) for low, high in zip(lb, ub)]
#np.random.seed(100)

# Define the objective function (sum of squared residuals)
def pump_equations(x):
    N2, D2, M2, P2 = x

    # Specific speed equation residual
    F1 = (N1 * np.sqrt(M1/rho1) / (g * P1/(rho1*g))**(3/4)) - (N2 * np.sqrt(M2/rho2) / (g * P2/(rho2*g))**(3/4))
    
    # Flow coefficient equation residual
    F2 = (M1/rho1 / (N1 * D1**3)) - (M2/rho2 / (N2 * D2**3))
    
    # Head coefficient equation residual
    F3 = (g * P1/(rho1*g) / (N1**2 * D1**2)) - (g * P2/(rho2*g) / (N2**2 * D2**2))
    
    # Reynolds number equation residual
    F4 = (rho1 * N1 * D1**2 / mu1) - (rho2 * N2 * D2**2 / mu2)
    
    # Sum of squared residuals
    #return F1**2 + F2**2 + F3**2 + F4**2
    return [F1, F2, F3, F4]

# Normalization function for better scaling
def normalize_residuals(residuals):
    # Scale each residual by a factor that brings them into a similar range
    scales = [1.0, 1.0e-3, 1.0e-3, 1]  # Example scales, adjust as needed
    return [residuals[i] / scales[i] for i in range(len(residuals))]

# Objective function for minimization (sum of squares of normalized residuals)
def objective_function(x):
    residuals = pump_equations(x)
    normalized_residuals = normalize_residuals(residuals)
    return sum([r**2 for r in normalized_residuals])


# Initial guesses for unknowns
x0 = np.array([3000, 80e-3, 0.2, 2e5])

# Perform optimization using 'SLSQP' (Sequential Least Squares Programming)
#result = sp.optimize.minimize(objective_function, x0, bounds=bounds, method='trust-constr', options={'disp': True, 'maxiter': 10000, 'xtol': 1e-16, 'gtol': 1e-16})
#result = sp.optimize.differential_evolution(objective_function, bounds, disp = True, popsize=300, tol = 1e-16, maxiter = 1000)
#result = sp.optimize.root(pump_equations, x0, method='trust-constr', bounds=bounds, options={'xtol': 1e-10, 'maxiter': 2000})

# Perform global optimization using basinhopping
# Basin Hopping Methods: Powell
result = sp.optimize.basinhopping(objective_function,x0=x0,minimizer_kwargs={"method": "Powell", "bounds": bounds},niter=4000,T=0.5,stepsize=1,disp=True)

basinhopping step 0: f 9.25127e-05
basinhopping step 1: f 9.25127e-05 trial_f 7.36579e+11 accepted 0  lowest_f 9.25127e-05
basinhopping step 2: f 2.38896e-10 trial_f 2.38896e-10 accepted 1  lowest_f 2.38896e-10
found new global minimum on step 2 with function value 2.38896e-10
basinhopping step 3: f 2.38896e-10 trial_f 7.12436e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 4: f 2.38896e-10 trial_f 7.34619e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 5: f 2.38896e-10 trial_f 7.39538e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 6: f 2.38896e-10 trial_f 2.38896e-10 accepted 1  lowest_f 2.38896e-10
found new global minimum on step 6 with function value 2.38896e-10
basinhopping step 7: f 2.38896e-10 trial_f 7.29202e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 8: f 2.38896e-10 trial_f 7.37271e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 9: f 2.38896e-10 trial_f 7.34547e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 10: f 4.30558e-08 tri

D:\anaconda3\Lib\site-packages\scipy\optimize\_basinhopping.py:303: OptimizeWarning: Initial guess is not within the specified bounds
  return self.minimizer(self.func, x0, **self.kwargs)
C:\Users\Krai\AppData\Local\Temp\ipykernel_21200\2382488813.py:22: RuntimeWarning: invalid value encountered in sqrt
  F1 = (N1 * np.sqrt(M1/rho1) / (g * P1/(rho1*g))**(3/4)) - (N2 * np.sqrt(M2/rho2) / (g * P2/(rho2*g))**(3/4))


basinhopping step 30: f 1.18042e-08 trial_f 7.39808e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 31: f 1.18042e-08 trial_f 7.36268e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 32: f 4.14645e-06 trial_f 4.14645e-06 accepted 1  lowest_f 2.38896e-10
basinhopping step 33: f 4.14645e-06 trial_f 7.37374e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 34: f 4.14645e-06 trial_f 7.37441e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 35: f 4.14645e-06 trial_f 7.34227e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 36: f 4.14645e-06 trial_f 7.39743e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 37: f 4.14645e-06 trial_f 7.40011e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 38: f 4.14645e-06 trial_f 7.39932e+11 accepted 0  lowest_f 2.38896e-10
basinhopping step 39: f 4.04814e-05 trial_f 4.04814e-05 accepted 1  lowest_f 2.38896e-10
basinhopping step 40: f 4.21242e-08 trial_f 4.21242e-08 accepted 1  lowest_f 2.38896e-10
basinhopping step 41:

In [2]:
N2, D2, M2, P2 = result.x[0], result.x[1], result.x[2], result.x[3]
Q2 = M2/rho2
specific_speed_1 = N1*np.sqrt(M1/rho1)/(P1/(rho1*g))**(3/4)
specific_speed_2 = N2*np.sqrt(M2/rho2)/(P2/(rho2*g))**(3/4)
print(f"Ns = {specific_speed_2:.4f}, Matching = {100-abs(specific_speed_1-specific_speed_2)/specific_speed_1*100} %")

flow_coeff_1 = Q1/N1/D1**3
flow_coeff_2 = Q2/N2/D2**3
print(f"Flow Coefficient Prototype = {flow_coeff_1}, Model = {flow_coeff_2}, Matching = {100-abs(flow_coeff_1-flow_coeff_2)/flow_coeff_1*100} %")

head_coeff_1 = g*P1/(rho1*g)/N1**2/D1**2
head_coeff_2 = g*P2/(rho2*g)/N2**2/D2**2
print(f"Head Coefficient Prototype = {head_coeff_1}, Model = {head_coeff_2}, Matching = {100-abs(head_coeff_1-head_coeff_2)/head_coeff_1*100} %")

Reysnold_1 = rho1*N1*D1**2/mu1
Reysnold_2 = rho2*N2*D2**2/mu2
print(f"Reynolds Number Prototype = {Reysnold_1:.4f}, Model = {Reysnold_2:.4f}, Matching = {100-abs(Reysnold_2-Reysnold_2)/Reysnold_1*100} %")

print(f"\nModel's Shaft Speed is {N2:.2f} RPM\nModel's Impeller Diameter is {D2*1e3:.2f} mm\nModel's Mass Flow Rate is {M2:.2f} Kg/s\nModel's Pressure Across Pump is {P2/1e5:.2f} Bar\nModel's Volumetric Flow Rate is {Q2*1000:.4f} L/s")

Ns = 11.3821, Matching = 99.9999999999506 %
Flow Coefficient Prototype = 3.2552083333333326e-05, Model = 3.2552083333336484e-05, Matching = 99.9999999999903 %
Head Coefficient Prototype = 0.000390625, Model = 0.00039062500000028246, Matching = 99.9999999999277 %
Reynolds Number Prototype = 7680000.0000, Model = 7680000.0000, Matching = 100.0 %

Model's Shaft Speed is 5500.00 RPM
Model's Impeller Diameter is 107.29 mm
Model's Mass Flow Rate is 0.26 Kg/s
Model's Pressure Across Pump is 1.58 Bar
Model's Volumetric Flow Rate is 0.2211 L/s
